Importamos las librerias

In [152]:
import pandas as pd
import numpy as np
import re

Creamos un data frame de base de datos.


In [153]:
df = pd.read_csv("../../IH_Shark_Attack_Correlation/input/attacks.csv",encoding='latin')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [154]:
#Vamos a ver su dimensión y completitud.
dim= df.shape
f"El documento tiene unas dimensiones de {dim}"

'El documento tiene unas dimensiones de (25723, 24)'

Examinamos como de completa es la tabla. Podemos observar como tenemos un 77% de valores nulos en los indices que más nos importan para nuestro estudio.(Activity, Fatal , Type)

In [155]:
missing_values= (df.isnull().sum()/df.shape[0]).sort_values(ascending=False) .head(10)
missing_values
#valid_columns = [colName for colName,isValid in valid_columns.items() if isValid]
#valid_columns = df.isnull().sum().sort_values(ascending=False) == 0
#type(valid_columns)

Unnamed: 22    0.999961
Unnamed: 23    0.999922
Time           0.885394
Species        0.865335
Age            0.865062
Sex            0.776970
Activity       0.776154
Location       0.775998
Fatal (Y/N)    0.775959
Area           0.772694
dtype: float64

Hipotesis: Los tiburones probocan menos muertes en caso de que se equivoquen de presa. Es decir, si confunden a un surfista con una tortuga estos no continuaran con el ataque hasta probocar la muerte. En cambio si atacan a un humano sabiendo que este es un humano le probocaran la muerte.

Escogemos las columnas que son relevantes para nuestro análisis, dejamos atras todas aquellas que tienen multitud de NULL values además de otras no relevantes para el estudio de nuestra hipotesis. 


In [156]:
Valid_colums =df[["Activity","Fatal (Y/N)","Type"]]
Valid_colums.head(30)
Valid_colums = Valid_colums.rename(columns={'Fatal (Y/N)':'Mortal',
                                   'Activity':'Deporte','Type':'Tipo'})
#Borramos las filas en las que no tengamos informacion en activity.                                                                                         

Borramos las filas en las que no tengamos informacion en activity, dado que según la actividad que realizaba la victima podremos determinar otras columnas, pero no inversamente.

In [157]:
def drop_null(table,subset):
    table.dropna(subset=[f"{subset}"],inplace=True)
    
    

In [158]:
Valid_colums.isnull().sum().sort_values(ascending=False)

Deporte    19965
Mortal     19960
Tipo       19425
dtype: int64

In [159]:
#Valid_colums = Valid_colums.dropna(subset=["Activity"])
drop_null(Valid_colums,"Activity")
Valid_colums.isnull().sum().sort_values(ascending=False)


KeyError: ['Activity']

Para validar nuestra hipotesis tambien debemos saber si el accidente fue Fatal o No, por lo que eliminaremos aquellas filas que no tengamos datos.

In [160]:
#Valid_colums = Valid_colums.dropna(subset=["Fatal (Y/N)"])
drop_null(Valid_colums,"Mortal")
print(Valid_colums.isnull().sum().sort_values(ascending=False))

Deporte    419
Tipo         3
Mortal       0
dtype: int64


In [161]:
Valid_colums = Valid_colums.dropna(subset=["Tipo"])
drop_null(Valid_colums,"Tipo")
print(Valid_colums.isnull().sum().sort_values(ascending=False))

Deporte    419
Tipo         0
Mortal       0
dtype: int64


<ipython-input-157-766211db099e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.dropna(subset=[f"{subset}"],inplace=True)


Vamos a renombrar las columnas para que sea mas entendible.

In [162]:
def standardize_data(df,column,old,new):
    Valid_colums[column] = Valid_colums[column].replace(old, new)

In [163]:
def c_data(column):
     print(Valid_colums[f"{column}"].value_counts())
    
    


In [164]:
c_data("Tipo")

Unprovoked      4589
Provoked         572
Sea Disaster     235
Boating          202
Boat             136
Invalid           23
Questionable       2
Boatomg            1
Name: Tipo, dtype: int64


In [165]:
standardize_data(Valid_colums,"Tipo","Sea Disaster","Unprovoked")
standardize_data(Valid_colums,"Tipo","Boat","Unprovoked")
standardize_data(Valid_colums,"Tipo","Boating","Unprovoked")
standardize_data(Valid_colums,"Tipo","Invalid","Unprovoked")
standardize_data(Valid_colums,"Tipo","Questionable","Unprovoked")
standardize_data(Valid_colums,"Tipo","Boatomg","Unprovoked")


<ipython-input-162-ce1d92d2a66d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Valid_colums[column] = Valid_colums[column].replace(old, new)


In [166]:
c_data("Tipo")

Unprovoked    5188
Provoked       572
Name: Tipo, dtype: int64


Finalmente, dado que para nuestro análisis no tendremos en cuenta los ataques probocados los eliminamos de nuestra muestra.

Al continuación debemos standarizar los datos entre deporte el cual el tiburon se cree que somos un otro animal, o sabe que somo un humano. Para ello, vamos a decir que cualquier deporte que implique unas aletas, tabla o remo el tiburon podría pensar que somos otra especie, en cambio si la victima es atacada y esta no portaba nada que pudiera dar a confusión podríamos decir que el tiburon quería atacar a un humano y el resultado podría ser fatal. 

In [1]:
dim1=Valid_colums.shape
(Valid_colums["Deporte"].value_counts()/dim1[0]).head(30)
                                                    

NameError: name 'Valid_colums' is not defined

In [168]:
#Solo por curiosidad mia veo que son Strings, por lo que voy a utilizar Reggex.:)
#for i in Valid_colums["Activity"]:
    #print(type(i))

In [169]:
print(Valid_colums)

           Deporte Mortal        Tipo
0         Paddling      N  Unprovoked
1         Standing      N  Unprovoked
2          Surfing      N  Unprovoked
3          Surfing      N  Unprovoked
4      Free diving      N    Provoked
...            ...    ...         ...
6297        Diving      Y  Unprovoked
6298  Pearl diving      Y  Unprovoked
6299      Swimming      Y  Unprovoked
6300           NaN      Y  Unprovoked
6301      Swimming      Y  Unprovoked

[5760 rows x 3 columns]


A continuación vamos a estandarizar los parametros de c_data, dado que los valores no estandares son residuales los sacaremos de nuestra muestra. 

In [170]:
c_data("Mortal")

N          4292
Y          1386
UNKNOWN      71
 N            7
M             1
2017          1
y             1
N             1
Name: Mortal, dtype: int64


In [171]:
Valid_colums = Valid_colums.rename(columns={'Fatal (Y/N)':'Mortal',
                                   'Activity':'Deporte','Type':'Tipo'})

In [172]:
drop_mortal= ['UNKNOWN', ' N','M','2017','y','N ']
Valid_colums= Valid_colums[~(Valid_colums["Mortal"].isin(drop_mortal))]
#Valid_colums.drop(index=Valid_colums[Valid_colums["Mortal"]=="UNKNOWN"], columns='Mortal')

In [173]:
c_data("Mortal")

N    4292
Y    1386
Name: Mortal, dtype: int64


In [174]:
Valid_colums.head(30)

,Deporte,Mortal,Tipo
0,Paddling,N,Unprovoked
1,Standing,N,Unprovoked
2,Surfing,N,Unprovoked
3,Surfing,N,Unprovoked
4,Free diving,N,Provoked
5,Kite surfing,N,Unprovoked
6,Swimming,Y,Unprovoked
7,Fishing,N,Unprovoked
8,Walking,N,Unprovoked
9,Standing,N,Unprovoked


In [2]:
activity_rename = {".*[Ss]urfing" : "Antonio",".*[Bb]oarding": "Paco"}
activity_rename={
".*[Ss]urfing":"Not Human Shape",
".*[Ss]wimming":"Human Shape",
".*[Ff]ishing":"Human Shape",
".*[Ss]pearfishing":"Human Shape",
".*[Bb]athing":"Human Shape",
".*[Ww]ading":"Human Shape",
".*[Dd]iving":"Human Shape",
".*[Ss]tanding":"Human Shape",
".*[Ss]norkeling":"Human Shape",
".*[Ss]cuba":"Human Shape",
".*[Bb]ody":"Not Human Shape",
".*[Bb]ody":"Not Human Shape",
".*[Ss]wimming":"Human Shape",
".*[Pp]earl":"Human Shape",
".*[Kk]ayaking":"Not Human Shape",
".*[Tt]reading":"Not Human Shape",
".*[Ff]ell":"Human Shape",
".*[Ff]ree":"Human Shape",
".*[Bb]oogie":"Not Human Shape",
".*[Ww]indsurfing":"Not Human Shape",
".*[Bb]oogie":"Not Human Shape",
".*[Ww]alking":"Human Shape",
".*[Ff]loating":"Human Shape",
".*[Ff]ishing":"Human Shape",
".*[Rr]owing":"Not Human Shape",
".*[Cc]anoeing":"Not Human Shape",
".*[Ss]urf":"Not Human Shape",
".*[Kk]ayak":"Not Human Shape"}

In [3]:
Shapes =[]
for i in Valid_colums.Deporte:
    for key,value in activity_rename.items():
        if re.search(key, str(i)):
            i=value
    Shapes.append(i)     
        
Shapes

NameError: name 'Valid_colums' is not defined

In [197]:
c_data("Deporte")

Surfing                                    927
Swimming                                   772
Fishing                                    408
Spearfishing                               305
Bathing                                    146
                                          ... 
Sitting                                      1
Fell overboard & rescuer                     1
Free diving  & spearfishing (submerged)      1
Wading (shrimping)                           1
Helmet diving, collecting trochus shell      1
Name: Deporte, Length: 1410, dtype: int64


In [200]:
Valid_colums.Deporte= Shapes

In [201]:
c_data("Deporte")

Antonio                                    1079
Swimming                                    772
Fishing                                     408
Spearfishing                                305
Bathing                                     146
                                           ... 
Fell oveboard from steamer Wanderer           1
Free diving  & spearfishing (submerged)       1
Wading (shrimping)                            1
Fell overboard from the steamship Chala       1
Helmet diving, collecting trochus shell       1
Name: Deporte, Length: 1326, dtype: int64
